In [19]:
import datetime
import pandas as pd

import pickle

from sklearn.feature_extraction.text import TfidfTransformer
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler

In [20]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [21]:
MODEL_FOLDER = "model"

PROCESSED_DATA_FOLDER = "data/4_all_data_preprocessed"
ISW_DATA_FILE = "all_isw.csv"
WEATHER_EVENTS_DATA_FILE = "features_generated_v1.csv"

OUTPUT_DATA_FILE = "all_features"

scaler_model = "scaler"
tfidf_transformer_model = "tfidf_transformer"
count_vectorizer_model = "count_vectorizer"
preciptype_encoder_model = "preciptype_encoder"
conditions_encoder_model = "conditions_encoder"

encoders_version = "v1"
scaler_version = "v1"
tfidf_transformer_version = "v4"
count_vectorizer_version = "v4"

In [22]:
def isNaN(num):
    return num != num

### Load preprocessed data

In [23]:
# Load CSVs
df_isw = pd.read_csv(f"{PROCESSED_DATA_FOLDER}/{ISW_DATA_FILE}", sep=";")
df_weather_events = pd.read_csv(f"{PROCESSED_DATA_FOLDER}/{WEATHER_EVENTS_DATA_FILE}", sep=";")

df_weather_events = df_weather_events.loc[:, ~df_weather_events.columns.str.contains('^Unnamed')]

print(df_isw.shape)

# Load TF-IDF
tfidf = pickle.load(open(f"{MODEL_FOLDER}/{tfidf_transformer_model}_{tfidf_transformer_version}.pkl", "rb"))
cv = pickle.load(open(f"{MODEL_FOLDER}/{count_vectorizer_model}_{count_vectorizer_version}.pkl", "rb"))

(402, 7)


### Prepare weather & events

In [24]:
# Technicaly it's feature engineering
print(df_weather_events['event_simultaneous_alarms'].max(), df_weather_events['event_simultaneous_alarms'].min())


58 0


In [25]:

# Replace timestamps with numerical values
df_weather_events.day_sunset = df_weather_events.day_sunset.apply(lambda x: 
    (datetime.datetime.strptime(x, "%H:%M:%S") - datetime.datetime.strptime("00:00:00", "%H:%M:%S")).total_seconds()
)
df_weather_events.day_sunrise = df_weather_events.day_sunrise.apply(lambda x: 
    (datetime.datetime.strptime(x, "%H:%M:%S") - datetime.datetime.strptime("00:00:00", "%H:%M:%S")).total_seconds()
)
df_weather_events.hour_datetime = df_weather_events.hour_datetime.apply(lambda x:
    (datetime.datetime.strptime(x, "%H:%M:%S") - datetime.datetime.strptime("00:00:00", "%H:%M:%S")).total_seconds()//3600
)

# Encode categories & save encoder
le = preprocessing.LabelEncoder()

df_weather_events.hour_conditions = le.fit_transform(df_weather_events.hour_conditions)
with open(f"{MODEL_FOLDER}/{conditions_encoder_model}_{encoders_version}.pkl", 'wb') as handle:
    pickle.dump(le, handle)

df_weather_events.hour_preciptype = le.fit_transform(df_weather_events.hour_preciptype)
with open(f"{MODEL_FOLDER}/{preciptype_encoder_model}_{encoders_version}.pkl", 'wb') as handle:
    pickle.dump(le, handle)

# Create new column with boolean event indicator
df_weather_events["event_indicator"] = df_weather_events.event_hour_level_event_datetimeEpoch.apply(lambda x: not isNaN(x))

# Drop now irrelevant features
features_to_drop = [
    "day_datetimeEpoch",
    "hour_datetimeEpoch",
    "event_all_region",
    "event_clean_end",
    "event_intersection_alarm_id",
    "event_start_time",
    "event_end_time",
    "event_start_hour",
    "event_end_hour",
    "event_day_date",
    "event_start_hour_datetimeEpoch",
    "event_end_hour_datetimeEpoch",	
    "event_hour_level_event_time",
    "event_hour_level_event_datetimeEpoch",
    "event_region_title",
    "event_region_city",
    "event_start",
    "event_end",
    "city",
    "region",
    "center_city_ua",
    "center_city_en",
    "region_alt"
]
df_weather_events = df_weather_events.drop(features_to_drop, axis=1)

# Convert everything to float
df_weather_events.hour_preciptype = df_weather_events.hour_preciptype.astype(float)
df_weather_events.hour_conditions = df_weather_events.hour_conditions.astype(float)
df_weather_events.region_id = df_weather_events.region_id.astype(float)
df_weather_events.event_alarms_past_24 = df_weather_events.event_alarms_past_24.astype(float)
df_weather_events.event_simultaneous_alarms = df_weather_events.event_simultaneous_alarms.astype(float)

df_weather_events.event_holiday_is_near = df_weather_events.event_holiday_is_near.apply(lambda x: 1.0 if x and not isNaN(x) else 0.0)

# Fill NAN
df_weather_events.fillna(0, inplace=True)

print(df_weather_events.shape)
print(df_weather_events.dtypes)
df_weather_events.sample(10)

(195084, 41)
city_resolvedAddress            object
day_datetime                    object
day_tempmax                    float64
day_tempmin                    float64
day_temp                       float64
day_dew                        float64
day_humidity                   float64
day_precip                     float64
day_precipcover                float64
day_solarradiation             float64
day_solarenergy                float64
day_uvindex                    float64
day_sunrise                    float64
day_sunset                     float64
day_moonphase                  float64
hour_datetime                  float64
hour_temp                      float64
hour_humidity                  float64
hour_dew                       float64
hour_precip                    float64
hour_precipprob                float64
hour_snow                      float64
hour_snowdepth                 float64
hour_preciptype                float64
hour_windgust                  float64
hour_windspe

,city_resolvedAddress,day_datetime,day_tempmax,day_tempmin,day_temp,day_dew,day_humidity,day_precip,day_precipcover,day_solarradiation,day_solarenergy,day_uvindex,day_sunrise,day_sunset,day_moonphase,hour_datetime,hour_temp,hour_humidity,hour_dew,hour_precip,hour_precipprob,hour_snow,hour_snowdepth,hour_preciptype,hour_windgust,hour_windspeed,hour_winddir,hour_pressure,hour_visibility,hour_cloudcover,hour_solarradiation,hour_solarenergy,hour_uvindex,hour_severerisk,hour_conditions,region_id,event_alarms_past_24,event_simultaneous_alarms,event_hours_from_last_alarm,event_holiday_is_near,event_indicator
12057,"Кропивницький, Україна",2022-07-27,31.9,14.6,23.2,14.0,60.0,0.000,0.00,302.8,26.2,9.0,19123.0,74130.0,0.94,22.0,23.2,59.96,15.0,0.0,0.0,0.0,0.0,5.0,5.4,2.9,84.7,1013.0,24.1,60.3,0.0,0.0,0.0,10.0,4.0,11.0,2.0,1.0,14.852222,0.0,False
80066,"Харків, Україна",2022-03-20,3.0,-10.0,-2.0,-9.5,57.8,0.000,0.00,214.0,18.6,6.0,20298.0,64066.0,0.58,2.0,-5.0,57.96,-12.0,0.0,0.0,0.0,5.5,5.0,7.2,7.2,80.0,1040.0,10.0,0.0,0.0,0.0,0.0,10.0,0.0,20.0,4.0,2.0,5.966944,0.0,True
27128,"Київ, Україна",2022-03-18,2.4,-6.3,-2.0,-11.3,50.7,0.000,0.00,195.6,16.9,5.0,21929.0,65246.0,0.51,23.0,-1.9,45.62,-12.1,0.0,0.0,0.0,0.0,5.0,5.4,0.0,3.2,1044.9,20.0,0.0,0.0,0.0,0.0,10.0,0.0,10.0,34.0,0.0,0.836944,0.0,True
63355,"Одеса, Україна",2022-05-10,18.2,10.9,14.9,3.3,48.3,0.000,0.00,324.8,28.2,9.0,19807.0,73061.0,0.30,1.0,15.1,75.49,10.8,0.0,0.0,0.0,0.0,5.0,24.1,12.6,17.3,1021.0,24.1,68.9,0.0,0.0,0.0,10.0,4.0,15.0,4.0,0.0,1.755556,0.0,True
6144,"Луцьк, Луцький район, Україна",2022-11-02,12.0,9.4,10.8,9.6,92.3,0.200,8.33,57.6,4.9,3.0,25898.0,60730.0,0.25,16.0,11.8,92.35,10.6,0.0,0.0,0.0,0.0,5.0,19.4,10.8,284.5,1018.0,24.1,100.0,38.0,0.1,0.0,10.0,3.0,3.0,0.0,0.0,5.281944,0.0,False
186479,"Суми, Україна",2022-12-30,3.1,0.2,1.5,-0.2,88.6,0.302,8.33,25.7,2.2,1.0,27823.0,56581.0,0.22,17.0,1.9,77.75,-1.6,0.0,0.0,0.0,2.9,5.0,19.4,14.4,181.0,1019.6,10.0,90.0,0.0,0.0,0.0,10.0,4.0,18.0,1.0,0.0,4.545278,1.0,False
137797,"Житомир, Україна",2023-01-17,6.2,1.3,4.2,3.0,90.0,3.000,4.17,17.0,1.4,1.0,28666.0,59602.0,0.83,22.0,3.8,93.84,2.9,0.0,0.0,0.0,0.5,5.0,18.0,9.0,174.5,1001.0,24.1,24.4,0.0,0.0,0.0,10.0,4.0,6.0,0.0,0.0,4.946667,0.0,False
175825,"Черкаси, Україна",2022-09-21,13.7,9.0,10.9,8.5,85.9,0.800,29.17,133.5,11.4,5.0,23780.0,67953.0,0.83,17.0,12.9,79.28,9.4,0.1,100.0,0.0,0.0,3.0,28.1,14.8,289.6,1012.0,24.1,81.1,266.0,1.0,3.0,10.0,7.0,23.0,1.0,0.0,3.045278,0.0,True
141302,"Тернопіль, Україна",2022-07-15,23.9,16.3,19.3,13.6,71.3,2.800,8.33,266.0,22.9,8.0,19624.0,76778.0,0.53,7.0,18.4,80.06,14.9,0.0,0.0,0.0,0.0,5.0,10.8,6.1,297.6,1011.0,24.1,77.2,56.0,0.2,1.0,10.0,4.0,19.0,1.0,0.0,18.783056,0.0,False
67277,"Одеса, Україна",2022-10-17,16.7,9.1,12.5,5.5,63.3,0.000,0.00,153.9,13.3,6.0,26212.0,65237.0,0.71,11.0,12.5,59.39,4.8,0.0,0.0,0.0,0.0,5.0,23.8,10.8,25.0,1034.0,24.1,0.0,443.0,1.6,4.0,10.0,0.0,15.0,4.0,2.0,1.076111,1.0,True


### Calculate TF-IDF matrix

In [16]:
# Calculate word count vector for each report (save as matrix)
word_count_vector = cv.transform(df_isw["lemm"].values.astype("U"))
# print("Word count vector:\n", word_count_vector)

# Calculate global tfidf vectors matrix
tfidf_vector = tfidf.transform(word_count_vector)

# transform tfidf matrix to sparse dataframe
df_tfidf_vector = pd.DataFrame.sparse.from_spmatrix(tfidf_vector, columns=cv.get_feature_names_out())
df_tfidf_vector = pd.concat([df_isw.date_tomorrow_datetime, df_tfidf_vector], axis=1)

print(df_tfidf_vector.shape)

(402, 719)


### Merge final dataset

In [18]:
# Merge weather events dataset with yesterday report tfidf matrix (takes 2m to execute)
df_final = df_weather_events.merge(df_tfidf_vector.add_prefix("isw_"),
                                    how="inner",
                                    left_on="day_datetime",
                                    right_on="isw_date_tomorrow_datetime")

# Normalize
scaler = StandardScaler()
df_float_values = df_final.drop(['event_indicator', 'city_resolvedAddress', 'day_datetime', 'isw_date_tomorrow_datetime'], axis=1)
float_values_scaled = scaler.fit_transform(df_float_values)
df_fload_values_scaled = pd.DataFrame(float_values_scaled, columns=df_float_values.columns)
df_final = pd.concat([df_final[['event_indicator', 'city_resolvedAddress', 'day_datetime', 'isw_date_tomorrow_datetime']], df_fload_values_scaled], axis=1)

# Save scaler
with open(f"{MODEL_FOLDER}/{scaler_model}_{scaler_version}.pkl", 'wb') as handle:
    pickle.dump(scaler, handle)

print(df_final.shape)
df_final.sample(5)

c:\Users\tmyro\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:768: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(
c:\Users\tmyro\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:768: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(


(192867, 760)


,event_indicator,city_resolvedAddress,day_datetime,isw_date_tomorrow_datetime,day_tempmax,day_tempmin,day_temp,day_dew,day_humidity,day_precip,day_precipcover,day_solarradiation,day_solarenergy,day_uvindex,day_sunrise,day_sunset,day_moonphase,hour_datetime,hour_temp,hour_humidity,hour_dew,hour_precip,hour_precipprob,hour_snow,hour_snowdepth,hour_preciptype,hour_windgust,hour_windspeed,hour_winddir,hour_pressure,hour_visibility,hour_cloudcover,hour_solarradiation,hour_solarenergy,hour_uvindex,hour_severerisk,hour_conditions,region_id,event_alarms_past_24,event_simultaneous_alarms,event_hours_from_last_alarm,event_holiday_is_near,isw_10km,isw_12km,isw_15km,isw_1st,isw_20km,isw_30km,isw_5km,isw_6km,isw_abandon,isw_abil,isw_abl,isw_access,isw_accus,isw_achiev,isw_across,isw_action,isw_activ,isw_ad,isw_addit,isw_address,isw_administr,isw_advanc,isw_advisor,isw_affect,isw_affili,isw_aim,isw_air,isw_airborn,isw_aircraft,isw_airstrik,isw_alexand,isw_allow,isw_almost,isw_along,isw_alongsid,isw_alreadi,isw_also,isw_although,isw_ammunit,isw_among,isw_amplifi,isw_annex,isw_announc,isw_anoth,isw_anti,isw_appar,isw_appeal,isw_appear,isw_appoint,isw_approxim,isw_april,isw_area,isw_arm,isw_armi,isw_armor,isw_around,isw_arrang,isw_arriv,isw_artilleri,isw_assault,isw_asset,isw_attempt,isw_author,isw_avail,isw_avdiivka,isw_avoid,isw_away,isw_axi,isw_back,isw_bakhmut,isw_bank,isw_base,isw_basi,isw_battalion,isw_battl,isw_becom,isw_began,isw_begin,isw_belaru,isw_belarusian,isw_belgorod,isw_bilohorivka,isw_blame,isw_border,isw_break,isw_bridg,isw_brigad,isw_build,isw_call,isw_campaign,isw_capabl,isw_captur,isw_carri,isw_case,isw_casualti,isw_cauldron,isw_caus,isw_center,isw_central,isw_challeng,isw_chang,isw_channel,isw_chechen,isw_chief,isw_child,isw_choos,isw_citizen,isw_civilian,isw_classifi,isw_clear,isw_click,isw_close,isw_colonel,isw_combat,isw_combin,isw_come,isw_command,isw_comment,isw_commerci,isw_commit,isw_commun,isw_compani,isw_complet,isw_compris,isw_concentr,isw_concern,isw_condemn,isw_condit,isw_confirm,isw_conflict,isw_connect,isw_conscript,isw_consid,isw_consist,isw_consolid,isw_construct,isw_contact,isw_contract,isw_control,isw_convent,isw_corp,isw_could,isw_council,isw_counter,isw_counterattack,isw_counteroffens,isw_countri,isw_cours,isw_cover,isw_creat,isw_crime,isw_crimea,isw_crimin,isw_critic,isw_cross,isw_current,isw_cut,isw_daili,isw_damag,isw_data,isw_date,isw_day,isw_decemb,isw_decis,isw_defend,isw_defens,isw_degrad,isw_demand,isw_demonstr,isw_deni,isw_deploy,isw_depot,isw_deputi,isw_describ,isw_despit,isw_destroy,isw_detail,isw_develop,isw_differ,isw_direct,isw_directli,isw_director,isw_discus,isw_disrupt,isw_district,isw_divis,isw_dmitri,isw_dnipro,isw_dnipropetrovsk,isw_dnr,isw_domest,isw_donba,isw_draw,isw_drive,isw_drone,isw_due,isw_duma,isw_earli,isw_east,isw_eastern,isw_econom,isw_effect,isw_eight,isw_eighteen,isw_eighti,isw_either,isw_element,isw_eleven,isw_emphas,isw_encircl,isw_end,isw_endnot,isw_energi,isw_enerhodar,isw_enforc,isw_engag,isw_enter,isw_entir,isw_entireti,isw_equip,isw_establish,isw_evacu,isw_evalu,isw_even,isw_evid,isw_exist,isw_expand,isw_expect,isw_explos,isw_express,isw_extens,isw_face,isw_facil,isw_fail,isw_failur,isw_fall,isw_fals,isw_famili,isw_far,isw_februari,isw_feder,isw_fifteen,isw_fifti,isw_fight,isw_fighter,isw_financi,isw_fire,isw_first,isw_five,isw_focu,isw_focus,isw_follow,isw_footag,isw_forcibl,isw_forecast,isw_foreign,isw_form,isw_format,isw_former,isw_forti,isw_fortif,isw_four,isw_fourteen,isw_front,isw_frontlin,isw_full,isw_fulli,isw_futur,isw_gain,isw_geneva,isw_geoloc,isw_geospati,isw_given,isw_gloc,isw_govern,isw_governor,isw_ground,isw_group,isw_grow,isw_guard,isw_gur,isw_haidai,isw_head,isw_heavi,isw_heavili,isw_held,isw_high,isw_highli,isw_highway,isw_himar,isw_hold,isw_hope,isw_hour,isw_howev,isw_hulyaipol,isw_human,isw_hundr,isw_hundred,isw_identifi,isw_imageri,isw_immedi,isw_immin,isw_import,isw_improv,isw_includ,isw_increas,isw_increasingli,isw_independ,isw_indic,isw_

In [222]:
# Save final dataset
with open(f"{PROCESSED_DATA_FOLDER}/{OUTPUT_DATA_FILE}.pkl", 'wb') as handle:
    pickle.dump(df_final, handle)